# Build the Classifier
This notebook is dedicated to creating classifiers and run classification analyses of interest on neuroimaging data.

Can we accurately classify:
- conditions across all data
- adults versus children across conditions
- adults versus children within each condition

Can we predict:
- child age based on multivariate patterns of brain activation
- child age based on brain activation in each condition

In [1]:
from pandas import DataFrame, Series, read_csv

# Study specific variables
study_home = '/home/camachocm2/Analysis/KidVid_MVPA'
standard_mask = study_home + '/template/MNI152_T1_2mm_brain_mask_KV.nii.gz'
gm_mask = study_home + '/template/gm_2mm_mask.nii.gz'
template = study_home + '/template/MNI152_T1_1mm_brain.nii.gz'
sub_data_file = study_home + '/doc/subjectinfo.csv'
preproc_dir = study_home + '/analysis/preproc/betas'
output_dir = study_home + '/analysis/classifier'

condition_data = read_csv(study_home + '/doc/conditionslist.csv')
subject_info = read_csv(sub_data_file, index_col=0)
subject_info.describe()

,age_mos,age_yrs,CBCL_intern,CBCL_extern,MAPDB_temploss,CBQ_Anger_Frustration,CBQ_Activity_Level,CBQ_Approach_Positive_Anticipation,CBQ_Attentional_ Focusing,CBQ_Discomfort,...,CBQ_Low_ Intensity_ Pleasure,CBQ_Perceptual_ Sensitivity,CBQ_Sadness,CBQ_Shyness,CBQ_Smiling_Laughter,age_mos_std,MAPDB_temploss_std,CBQ_Anger_Frustration_std,CBCL_intern_std,CBCL_extern_std
count,51.000000,51.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01,3.000000e+01
mean,186.007843,15.137255,5.233333,5.366667,12.233333,4.541270,3.883444,4.994555,4.933222,4.022220,...,5.258333,5.094445,3.971604,3.716889,5.377889,6.666682e-12,-3.333334e-12,-3.333319e-12,3.333327e-12,3.333339e-12
std,119.838533,10.282061,5.969366,6.183869,7.933705,1.017132,0.981753,0.957611,1.168547,1.190428,...,1.398095,1.252341,1.068094,1.450841,1.171197,1.017095e+00,1.017095e+00,1.017095e+00,1.017095e+00,1.017095e+00
min,58.000000,4.000000,0.000000,0.000000,0.000000,2.285714,2.000000,2.833333,2.833333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-1.648830e+00,-1.568304e+00,-2.255473e+00,-8.916857e-01,-8.826855e-01
25%,86.150000,6.000000,1.000000,1.250000,5.250000,3.857143,3.083333,4.208333,4.041667,3.500000,...,4.562500,4.666667,3.750000,2.667500,5.000000,-8.812385e-01,-8.952583e-01,-6.841020e-01,-7.212999e-01,-6.770911e-01
50%,122.100000,10.000000,3.000000,4.000000,11.500000,4.500000,3.668333,5.000000,5.000000,4.000000,...,5.625000,5.333333,4.142857,3.916667,5.583333,-9.481084e-02,-9.401280e-02,-4.126833e-02,-3.805283e-01,-2.247833e-01
75%,288.000000,24.000000,6.750000,7.750000,16.750000,5.178571,4.666667,5.833333,5.958333,4.791667,...,6.218750,5.791667,4.678571,4.833333,6.000000,8.411323e-01,5.790334e-01,6.372783e-01,2.584185e-01,3.920001e-01
max,528.000000,44.000000,24.000000,31.000000,31.000000,6.428571,5.833333,6.670000,7.000000,6.000000,...,7.000000,7.000000,5.000000,5.500000,6.833333,1.807685e+00,2.405873e+00,1.887233e+00,3.197574e+00,4.216057e+00


## Step 1: Create feature set and labels

In [2]:
## Create a conditions list for the feature set
condition_labels = condition_data['labels'].tolist()
subjects_list = subject_info['subjID'].tolist()
age_group_list = subject_info['group'].tolist()
ages_mos_list = subject_info['age_mos'].tolist()
mapdb_temploss_list = subject_info['MAPDB_temploss_std'].tolist()
cbq_angfrust_list = subject_info['CBQ_Anger_Frustration_std'].tolist()

conditions = condition_data
conditions['subject'] = Series(subjects_list[0], index=conditions.index)
conditions['ageGroup'] = Series(age_group_list[0], index=conditions.index)
conditions['age'] = Series(ages_mos_list[0], index=conditions.index)
conditions['MAPDB'] = Series(mapdb_temploss_list[0], index=conditions.index)
conditions['CBQ'] = Series(cbq_angfrust_list[0], index=conditions.index)


for a in range(1,len(subjects_list)):
    temp=DataFrame()
    temp['labels'] = Series(condition_labels)
    temp['subject'] = Series(subjects_list[a], index=temp.index)
    temp['ageGroup'] = Series(age_group_list[a], index=temp.index)
    temp['age'] = Series(ages_mos_list[a], index=temp.index)
    temp['MAPDB'] = Series(mapdb_temploss_list[a], index=temp.index)
    temp['CBQ'] = Series(cbq_angfrust_list[a], index=temp.index)
    
    conditions = conditions.append(temp, ignore_index=True)

#conditions.to_csv(output_dir + '/featureset_key.csv')
conditions.describe()

,age,MAPDB,CBQ
count,1224.000000,7.200000e+02,7.200000e+02
mean,186.007843,-3.333353e-12,-3.333294e-12
std,118.706330,1.000695e+00,1.000695e+00
min,58.000000,-1.568304e+00,-2.255473e+00
25%,84.500000,-9.273081e-01,-6.841020e-01
50%,122.100000,-9.401280e-02,-4.126833e-02
75%,288.000000,6.110832e-01,7.444172e-01
max,528.000000,2.405873e+00,1.887233e+00


In [3]:
## Concatenate all the parameter estimates from preproc to create a feature set
from glob import glob
from nipype.interfaces.fsl.utils import Merge
files = glob(preproc_dir + '/*/betas.nii.gz')
files = sorted(files)

bold_feature_data = output_dir + '/featureset.nii.gz'

merge = Merge()
merge.inputs.in_files = files
merge.inputs.dimension = 't'
merge.inputs.merged_file = bold_feature_data
#merge.run()

In [4]:
# determine which analysis to run
analysis = 'all_conditions'

if analysis == 'all_conditions':
    mask = conditions['labels'].isin(['negative','positive','neutral'])
    labels = conditions['labels']
    type_svm = 'binary'
elif analysis == 'allConds_predAge':
    mask = conditions['labels'].isin(['negative','positive','neutral'])
    labels = conditions['ageGroup']
    type_svm = 'binary'
elif analysis == 'negative':
    mask = conditions['labels'].isin(['negative'])
    labels = conditions['ageGroup']
    type_svm = 'binary'
elif analysis == 'positive':
    mask = conditions['labels'].isin(['positive'])
    labels = conditions['ageGroup']
    type_svm = 'binary'
elif analysis == 'neutral':
    mask = conditions['labels'].isin(['neutral'])
    labels = conditions['ageGroup']
    type_svm = 'binary'
elif analysis=='age':
    mask = conditions['ageGroup']=='child'
    labels = conditions['age']
    type_svm = 'nonbinary'
elif analysis == 'age_neg':
    mask = (conditions['ageGroup']=='child') & (conditions['labels']=='negative')
    labels = conditions['age']
    type_svm = 'nonbinary'
elif analysis == 'age_pos':
    mask = (conditions['ageGroup']=='child') & (conditions['labels']=='positive')
    labels = conditions['age']
    type_svm = 'nonbinary'
elif analysis == 'age_neu':
    mask = (conditions['ageGroup']=='child') & (conditions['labels']=='neutral')
    labels = conditions['age']
    type_svm = 'nonbinary'

results_file = open(output_dir + '/results_' + analysis + '_final.txt','w')
labels[mask].describe()

count         1224
unique           3
top       negative
freq           408
Name: labels, dtype: object

## Support Vector Classification

The below cells perform categorical classification

In [ ]:
if type_svm == 'binary':
    # Perform the support vector classification
    from nilearn.input_data import NiftiMasker
    from sklearn.svm import SVC
    from sklearn.feature_selection import f_classif, SelectPercentile
    from sklearn.pipeline import Pipeline

    # Set up the support vector classifier
    svc = SVC(kernel='linear')
    masker = NiftiMasker(mask_img=gm_mask,standardize=True, 
                         memory='nilearn_cache', memory_level=1)
    
    # Select the features contributing to the model
    feature_selection = SelectPercentile(f_classif, percentile=5) #0.05/228453 voxels
    fs_svc = Pipeline([('feat_select', feature_selection), ('svc', svc)])

    # Run the classifier
    X = masker.fit_transform(bold_feature_data)
    X = X[mask]
    maskedlabels=labels[mask]
    fs_svc.fit(X, maskedlabels)
    
    # Obtain prediction values via cross validation
    from sklearn.model_selection import cross_validate, LeaveOneGroupOut, cross_val_predict

    loso = LeaveOneGroupOut()
    cv_scores = cross_validate(fs_svc, X, y=maskedlabels, n_jobs=10, return_train_score=True,
                               groups=conditions['subject'][mask], cv=loso, scoring='accuracy')
    y_pred = cross_val_predict(fs_svc, X, y=maskedlabels, n_jobs=10,
                               groups=conditions['subject'][mask], cv=loso)
    
    ## Save the SVM weights to a nifti
    coef = svc.coef_
    coef = feature_selection.inverse_transform(coef)
    weight_img = masker.inverse_transform(coef)
    weight_img.to_filename(output_dir + '/svmweights_'+ analysis +'.nii.gz')
    
    ## Calculate performance metrics
    from sklearn.metrics import recall_score, precision_score
    
    classification_accuracy = cv_scores['test_score'].mean()
    chance = 1. / len(labels.unique())
    print("Classification accuracy: %.4f / Chance level: %f" % 
          (classification_accuracy, chance))
    
    for label in maskedlabels.unique():
        sensitivity = recall_score(maskedlabels,y_pred,labels=[label],average='weighted')
        precision = precision_score(maskedlabels,y_pred,labels=[label],average='weighted')
        
        results_file.write("%s: classification accuracy: %.4f \n chance level: %f \n sensitivity: %f \n precision: %f \n" % 
        (label, classification_accuracy, chance, sensitivity, precision))
    
    # compute and display a confusion matrix
    from sklearn.metrics import confusion_matrix
    from numpy import set_printoptions
    import itertools
    import matplotlib.pyplot as plt

    cnf_matrix = confusion_matrix(maskedlabels, y_pred)
    set_printoptions(precision=2)
    classes = maskedlabels.unique()

    def plot_confusion_matrix(cm, classes):
        from numpy import arange
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Confusion matrix')
        plt.colorbar()
        tick_marks = arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45, size=16)
        plt.yticks(tick_marks, classes, size=16)

        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j],  'd'),
                     horizontalalignment='center',
                     color='white' if cm[i, j] > thresh else 'black', size=16)

        plt.tight_layout()
        plt.ylabel('True label', size=16)
        plt.xlabel('Predicted label', size=16)

    plot_confusion_matrix(cnf_matrix, classes)
    plt.savefig(output_dir + '/confusion_matrix_' + analysis + '.svg', transparent=True)
    plt.close()
    
    results_file.close()

### Perform permutation testing to get a p-value for the classifier

In [ ]:
'''
N.B.: in order to use the below function (permutation_test_score) for this particular analysis, 
I added the following code to the _shuffle function (starts on line 966 of sklearn/model_selection/_validation.py) code:

     elif permute_groups==True:
        indices = np.arange(len(groups))
        indices = random_state.permutation(indices)

and I added an argument to the permutation_test_score function (permute_groups=True) that is passed to the _shuffle function. 
This enables groups to be used for cross validation, but ignores groups for permutation. This is useful if you have multiple
features per subject with the same label and you are using grouping to denote a whole subject for LOSO cross validation.
'''

In [ ]:
from sklearn.model_selection import permutation_test_score
import matplotlib.pyplot as plt
from numpy import savetxt

results_file = open(output_dir + '/permut_results_' + analysis + '_final.txt','w')

if type_svm == 'binary':
    # Perform permutation testing to get a p-value
    score, permutation_scores, pvalue = permutation_test_score(fs_svc, X, maskedlabels, scoring='accuracy', 
                                                               cv=loso, n_permutations=500, n_jobs=10, 
                                                               groups=conditions['subject'][mask], permute_groups=True)
    savetxt(output_dir + '/permutation_scores_' + analysis + '.txt', permutation_scores)

    print("Classification score %s (pvalue : %s)" % (score, pvalue))
    # Save a figure of the permutation scores
    plt.hist(permutation_scores, 20, label='Permutation scores',
             edgecolor='black')
    ylim = plt.ylim()
    plt.plot(2 * [score], ylim, '--g', linewidth=3,
             label='Classification Score (pvalue %f)' % pvalue)
    plt.plot(2 * [1. / len(labels.unique())], ylim, '--k', linewidth=3, label='Luck')
    plt.ylim(ylim)
    plt.legend()
    plt.xlabel('Score')
    plt.savefig(output_dir + '/permutation_plot_' + analysis + '.svg', transparent=True)
    plt.close()
    
    # save final pval/classifier score
    results_file.write("Classification score %s (pvalue : %s)" % (score, pvalue))
    results_file.close()

## Support Vector Regression

The below cells performs continuous classification (i.e. predict a continuous variable) based on age.

In [ ]:
if type_svm == 'nonbinary':
    # Perform the support vector classification
    from nilearn.input_data import NiftiMasker
    from sklearn.feature_selection import f_regression, SelectPercentile
    from sklearn.svm import SVR
    from sklearn.pipeline import Pipeline

    # Set up the regression
    svr = SVR(kernel='linear', C=1)
    masker = NiftiMasker(mask_img=gm_mask,standardize=True, 
                         memory='nilearn_cache', memory_level=1)
    
    feature_selection = SelectPercentile(f_regression, percentile=5)
    fs_svr = Pipeline([('feat_select', feature_selection), ('svr', svr)])
    
    # Run the regression
    X = masker.fit_transform(bold_feature_data)
    X = X[mask]
    maskedlabels=labels[mask]
    fs_svr.fit(X, maskedlabels)
        
    from sklearn.model_selection import cross_val_predict, LeaveOneGroupOut

    loso = LeaveOneGroupOut()
    y_pred = cross_val_predict(fs_svr, X, y=maskedlabels, n_jobs=6,
                               groups=conditions['subject'][mask],cv=loso)
    # save weights
    coef = svr.coef_
    coef = feature_selection.inverse_transform(coef)
    coef_image = masker.inverse_transform(coef)
    coef_image.to_filename(output_dir + '/svrweights_' + analysis + '.nii.gz')
    
    ages_df = conditions[mask]
    ages_df['pred_age'] = Series(y_pred, index=ages_df.index)
    ages_df.head()

    actual_age = ages_df.groupby(['subject'])['age'].mean()
    actual_age = actual_age.tolist()
    pred_age = ages_df.groupby(['subject'])['pred_age'].mean()
    pred_age = pred_age.tolist()

    from scipy.stats import linregress
    slope, intercept, r_val, p_val, stderr = linregress(actual_age, pred_age) 

    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(actual_age, pred_age)
    
    from scipy.stats import spearmanr
    spear_r, spear_p = spearmanr(actual_age, pred_age)

    print("prediction accuracy: %.4f / p-value: %f / MSE: %f // Spearman: %f / p-value: %f" % (r_val, p_val, mse, spear_r, spear_p))

    # plot the predicted versus actual values
    import matplotlib.pyplot as plt
    plt.scatter(actual_age, pred_age, color='b')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(output_dir + '/scatter_pred_actual_mean_' + analysis + '_final.svg', transparent=True)
    plt.show()
    plt.close()

    results_file.write("MEAN prediction accuracy r-value: %.4f / p-value: %f / MSE: %f // Spearman: %f / p-value: %f \n" % (r_val, p_val, mse, spear_r, spear_p))
    results_file.write('predicted: ' + str(pred_age) + '\n')
    results_file.write('actual: ' + str(actual_age) + '\n')


    actual_age = ages_df.groupby(['subject'])['age'].median()
    actual_age = actual_age.tolist()
    pred_age = ages_df.groupby(['subject'])['pred_age'].median()
    pred_age = pred_age.tolist()

    from scipy.stats import linregress
    slope, intercept, r_val, p_val, stderr = linregress(actual_age, pred_age) 
    
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(actual_age, pred_age)

    from scipy.stats import spearmanr
    spear_r, spear_p = spearmanr(actual_age, pred_age)

    print("prediction accuracy: %.4f / p-value: %f / MSE: %f // Spearman: %f / p-value: %f" % (r_val, p_val, mse, spear_r, spear_p))

    # plot the predicted versus actual values
    import matplotlib.pyplot as plt
    plt.scatter(actual_age, pred_age, color='b')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(output_dir + '/scatter_pred_actual_median_' + analysis + '_final.svg', transparent=True)
    plt.close()
    
    results_file.write("MEDIAN prediction accuracy r-value: %.4f / p-value: %f / MSE: %f // Spearman: %f / p-value: %f \n" % (r_val, p_val, mse, spear_r, spear_p))
    results_file.write('predicted: ' + str(pred_age) + '\n')
    results_file.write('actual: ' + str(actual_age) + '\n')
    
    results_file.close()

### Perform permutation testing

In [ ]:
if type_svm == 'nonbinary':
    ## Perform permutation testing to get a p-value for MSE
    score, permutation_scores, pvalue = permutation_test_score(fs_svr, X, maskedlabels, scoring='neg_mean_squared_error', 
                                                               cv=loso, n_permutations=500, n_jobs=10, 
                                                               groups=conditions['subject'][mask], permute_groups=True)
    savetxt(output_dir + '/permutation_scores_mse_' + analysis + '.txt', permutation_scores)

    # Save a figure of the permutation scores
    plt.hist(permutation_scores, 20, label='Permutation scores',
             edgecolor='black')
    ylim = plt.ylim()
    plt.plot(2 * [score], ylim, '--g', linewidth=3,
             label='Mean Squared Error (pvalue %f)' % pvalue)
    plt.ylim(ylim)
    plt.legend()
    plt.xlabel('Score')
    plt.savefig(output_dir + '/permutation_plot_mse_' + analysis + '.svg', transparent=True)
    plt.close()

    # save final pval/classifier score
    results_file.write('MSE score %s (pvalue : %s) \n' % (score, pvalue))
    
    ## Perform permutation testing to get a p-value for r-squared
    score, permutation_scores, pvalue = permutation_test_score(fs_svr, X, maskedlabels, scoring='r2', 
                                                               cv=loso, n_permutations=500, n_jobs=10, 
                                                               groups=conditions['subject'][mask], permute_groups=True)
    savetxt(output_dir + '/permutation_scores_r2_' + analysis + '.txt', permutation_scores)

    # Save a figure of the permutation scores
    plt.hist(permutation_scores, 20, label='Permutation scores',
             edgecolor='black')
    ylim = plt.ylim()
    plt.plot(2 * [score], ylim, '--g', linewidth=3,
             label='R-squared (pvalue %f)' % pvalue)
    plt.ylim(ylim)
    plt.legend()
    plt.xlabel('Score')
    plt.savefig(output_dir + '/permutation_plot_r2_' + analysis + '.svg', transparent=True)
    plt.close()

    # save final pval/classifier score
    results_file.write('R square: %s (pvalue : %s) \n' % (score, pvalue))
    results_file.close()    

In [2]:
from glob import glob
from numpy import mean, std, loadtxt
files = glob('/home/camachocm2/Analysis/KidVid_MVPA/analysis/classifier/final_SVM_linear_5percent/permutation_scores_*.txt')
for file in files:
    analysis = file[103:-4]
    scores = loadtxt(file)
    print(analysis + ' average = ' + str(mean(scores)) + ', SD = ' + str(std(scores)))

neutral average = 0.5111225490196079, SD = 0.03207026651627986
negative average = 0.510563725490196, SD = 0.03202352909661293
positive average = 0.510892156862745, SD = 0.03239365975956134
allConds_predAge average = 0.5079248366013073, SD = 0.017480338393142128
all_conditions average = 0.331312091503268, SD = 0.014934348315243226
